In [1]:
import redis

import time
from multiprocessing import Process

redis_main = redis.Redis(host='localhost', port=6379)
redis_s2 = redis.RedisCluster(host='localhost', port=7000)



def conn(serv,serv2,s):
    i = 1
    Name = i
    while Name<i+10:
        if (serv.hget(Name-1, "Hash") != serv.hget(Name, "Hashnext")):
            print("ОШИБКА НАЧАЛАСЬ ИНЦИНАЛИЗАЦИЯ ПРОВЕРКИ БЛОКОВ", Name, s)
            resiv(Name, serv, serv2)
        Name+1
    if serv.hget(Name+1,"Hash") != None:
        i+=1

def resiv(i,fs,ss):
    fs.hset(i, key="Out",   value=ss.hget(i, "Out"))
    fs.hset(i, key="Num",  value=ss.hget(i, "Num"))
    fs.hset(i, key="In",   value=ss.hget(i, "In"))
    fs.hset(i, key="Hash", value=ss.hget(i, "Hash"))
    fs.hset(i, key="Hashnext", value=ss.hget(i, "Hashnext"))



def startcheck():
    tm=Process(target=conn(redis_s2, redis_main, "s2")).start()
    ###запуск проверки целосности2го сервера
    ts=Process(target=conn(redis_main, redis_s2, "main")).start()

In [2]:
import redis

import pickle
import socket

redis_main = redis.RedisCluster(host='127.0.0.1', port=7000,)
#redis_s2 = redis.Redis(host='192.168.0.40', port=7003, db=0)
#redis_main.flushall()
#redis_s2.flushall()


In = "input"
Out = "output"
Num = "Number"
Name=0

BUFFER_SIZE = 4096

server = socket.socket()
server.bind(('', 9090))
server.listen(1)




def conn(Name):
    i = 0
    while True:
        conn, addr = server.accept()
        print('Connected:', addr)
        adate = bytearray()

        while True:
            date = conn.recv(BUFFER_SIZE)
            if not date:
                break

            adate += date
        obj = pickle.loads(adate)
        print('Obj:', obj)
        Out = obj.pop(0)
        Num = obj.pop(0)
        In = obj.pop(0)
        Hash = hash(Out+str(Num)+In)

        pool(redis_main, Name, Out, Num, In, Hash,"serv1")
        #pool(redis_s2, Name, Out, Num, In, Hash,"serv2")
        Name += 1
        print('Отправитель: ', Out, ' Кол-во: ', Num, ' Получатель: ', In, ' Hash: ', Hash)
        conn.close()

def pool(redis, Name, Out, Num, In, Hash,ser):

    redis.hset(Name, key="Out", value=Out)
    redis.hset(Name, key="Num", value=Num)
    redis.hset(Name, key="In", value=In)
    redis.hset(Name, key="Hash", value=Hash)

    if Name != 0:
        redis.hset(Name, key="Hashnext", value=(redis_main.hget(Name-1, "Hash")))
    else:
        redis.hset(Name, key="Hashnext", value=0)
    return ()

conn(Name)

Connected: ('127.0.0.1', 55240)
Obj: ['ho311g', 343334, 'jgiffg']
Отправитель:  ho311g  Кол-во:  343334  Получатель:  jgiffg  Hash:  -7440365577972435790
Connected: ('127.0.0.1', 55242)
Obj: ['ho311g', 343334, 'jgiffg']
Отправитель:  ho311g  Кол-во:  343334  Получатель:  jgiffg  Hash:  -7440365577972435790
Connected: ('127.0.0.1', 55246)
Obj: ['ho311g', 343334, 'jgiffg']
Отправитель:  ho311g  Кол-во:  343334  Получатель:  jgiffg  Hash:  -7440365577972435790
